
# Create a ReAct Agent

In this, we will create a virtual stock advisor agent that uses the **ReAct** prompting framework. This agent will reason and act based on the prompts and tools provided.

Of course, the agent is built for demonstration purposes and the answers shouldn't be used as investment advice!



In [0]:
%pip install -U -qq langchain-databricks langchain==0.3.7 langchain_community==0.3.5 wikipedia==1.4.0 youtube-search duckduckgo-search

dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
petastorm 0.12.1 requires pyspark>=2.1.0, which is not installed.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires numpy<1.24,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.11.7 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ../Includes/Classroom-Setup-03LAB

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


30
30



The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


In [0]:
import mlflow
mlflow.langchain.autolog()

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser10993843_1753249626@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser10993843_1753249626
Working Directory: /Volumes/dbacademy/ops/labuser10993843_1753249626@vocareum_com
Dataset Location:  NestedNamespace (dais='/Volumes/dbacademy_dais/v01', docs='/Volumes/dbacademy_docs/v01')


## Step 1: Define the Brain of the Agent

Let's start by defining the agent brain using one of the available LLMs on Databricks.

* For this lab, use **databricks-meta-llama-3-3-70b-instruct** 

* Define the maximum number of tokens

In [0]:
from langchain.agents import AgentType, initialize_agent
from langchain_databricks import ChatDatabricks

## define the brain
llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)

/root/.ipykernel/21002/command-3772737368744671-465907747:5: LangChainDeprecationWarning: Use databricks_langchain.ChatDatabricks
  llm_llama = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct", max_tokens = 2500)


In [0]:
## let's test the brain
llm_llama.invoke("Hi! How are you?")

AIMessage(content="Hello! I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have! How about you? How's your day going so far?", additional_kwargs={}, response_metadata={'prompt_tokens': 16, 'completion_tokens': 52, 'total_tokens': 68}, id='run--71acc9b6-db1b-4f74-9b12-8f81e97d9c6c-0')

Trace(request_id=tr-e7ed6add1cfa4879bd79a89bce0f2eba)

## Step 2 : Define Tools that the Agent Will Use

For an agent, the toolset is a crucial component to complete the defined task. The agent will reason about the next action to take and use an appropriate tool to complete that action.

For our stock advisor agent, we will use the following tools:

* **Wikipedia tool**: For searching company details. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/wikipedia/)]

* **DuckDuckGo Search tool**: This will be used for searching the latest news and updates about a stock.[[LangChain Doc](https://python.langchain.com/docs/integrations/tools/ddg/)]

* **YouTube search tool**: This will be used to search for review videos about the latest financial reports. [[LangChain Doc](https://python.langchain.com/docs/integrations/tools/youtube/)]

### Wikipedia Tool

In [0]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)
tool_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [0]:
## Test the Wikipedia tool
tool_wiki.run("Apple company history")

"Page: Apple Inc.\nSummary: Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services. Founded in 1976 as Apple Computer Company by Steve Jobs, Steve Wozniak and Ronald Wayne, the company was incorporated by Jobs and Wozniak as Apple Computer, Inc. the following year. It was renamed Apple Inc. in 2007 as the company had expanded its focus from computers to consumer electronics. Apple is the largest technology company by revenue, with US$391.04 billion in the 2024 fiscal year.\nThe company was founded to produce and market Wozniak's Apple I personal computer. Its second computer, the Apple II, became a best seller as one of the first mass-produced microcomputers. Apple introduced the Lisa in 1983 and the Macintosh in 1984, as some of the first computers to use a graphical user interface and a mouse. By 1985, internal company problems led to Jo

Trace(request_id=tr-aa596ab1861840a494efcdb1548978fe)

### DuckDuckGo Search Tool

In [0]:
## DuckDuckGo Search Tool
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

tool_news = DuckDuckGoSearchRun()

In [0]:
## Test DuckDuckGo tool
tool_news.run("Latest news about Apple stock")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


'5 days ago · Apple Inc. (NASDAQ:AAPL) shares are reacting to a mix of analyst upgrades, competitive talent shifts, … 2 days ago · After Apple’s recent underperformance versus Big Tech companies, some are calling upon the … Jul 16, 2025 · Get the latest news and real-time alerts from Apple Inc. (AAPL) stock at Seeking Alpha. Jun 26, 2025 · Check this page for Apple stock news as well as how the rest of the world reacts to Apple’s next move in … 4 days ago · Get the latest news on Apple stock with updates on market trends, financials, and analysis from Benzinga. …'

Trace(request_id=tr-f6e2e9002ef84445a41f150b54861b49)

### YouTube Search Tool

In [0]:
from langchain_community.tools import YouTubeSearchTool

tool_youtube = YouTubeSearchTool()

<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


In [0]:
## Test the YouTube search tool 
tool_youtube.run("Apple company stock")

<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


"['https://www.youtube.com/watch?v=T1X0i_TjTtk&pp=ygUTQXBwbGUgY29tcGFueSBzdG9jaw%3D%3D', 'https://www.youtube.com/watch?v=XgOvWDwg5zU&pp=ygUTQXBwbGUgY29tcGFueSBzdG9jaw%3D%3D']"

Trace(request_id=tr-3a36d33347f64c0091b23d7da95969df)

### Define Toolset

Next, add all tools together to be used in the next step.

In [0]:
tools = [tool_wiki, tool_youtube, tool_news]

## Step 3: Define Agent Logic

An agent requires a planning prompt that defines how it should **reason and act**. LangChain provides a few ready-to-use prompt templates through its **[LangChain Hub](https://smith.langchain.com/hub)**, including those that follow the **ReAct (Reason + Act)** style.

In this task, you'll pull a pre-defined ReAct-style prompt from LangChain Hub, which is designed to work with tool-using agents.

> 💡 Using the Hub simplifies your code and ensures you're working with prompt templates that are community-tested and regularly maintained.

📚 [Learn more: LangChain Agent Executor Documentation](https://python.langchain.com/docs/how_to/agent_executor/#create-the-agent)

In [0]:
## Import LangChain Hub
from langchain import hub

## Pull a ReAct-style prompt for function/tool-calling agents
prompt = hub.pull("hwchase17/openai-functions-agent")

## Display the prompt structure
prompt.messages

<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Step 4: Create the Agent

The final step is to create the agent using the LLM, planning prompt and toolset.

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm_llama, tools, prompt)
agent_exec  = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Step 5: Run the Agent

Let's test the agent!

Note: YouTube search might not return the best results. For better results, you could use a paid API/Service that supports better search.

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

Format the final answer as HTML.
</blockquote>


In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

    Format the final answer as HTML.
    """})

displayHTML(response["output"])



> Entering new AgentExecutor chain...


<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: ImportHookFinder.find_spec() not found; falling back to find_module()


<p>Investing in Apple stock can be a viable option for those looking to diversify their portfolio with a well-established technology company. Apple is known for its innovative products and strong brand loyalty, which can contribute to its financial stability and potential for growth. However, it's essential to stay informed about the latest news and trends affecting the company.</p>

<p>To get an overview of the latest news about Apple, let's check what's currently available on DuckDuckGo:</p>
<function=duckduckgo_search>{"query": "Apple latest news)}</function>

> Finished chain.


Investing in Apple stock can be a viable option for those looking to diversify their portfolio with a well-established technology company. Apple is known for its innovative products and strong brand loyalty, which can contribute to its financial stability and potential for growth. However, it's essential to stay informed about the latest news and trends affecting the company. 

 To get an overview of the latest news about Apple, let's check what's currently available on DuckDuckGo: 
 {"query": "Apple latest news)}

Trace(request_id=tr-deaa4491669d474c90ed81bb1c23a9e1)

It seems like the agent uses company name only when searching for videos. **Let's play with the prompt to get better results.**

In real-world, the agent would be used in a conversational way and we would just chat with the agent to tweak the search query or to adjust the response in a desired format. 

**Use this prompt:**
<blockquote>
What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.

When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

Write a nice summary of the results.

Format the final answer as HTML.
</blockquote>

In [0]:
response = agent_exec.invoke({"input": 
    """
    What do you think about investing in Apple stock? Provide an overview of the latest news about the company. Next, search a video on YouTube which reviews Apple's latest financials.
    When searching on YouTube use such as "Apple  stock finance investment earnings valuation".

    Write a nice summary of the results.

    Format the final answer as HTML.

    """})
displayHTML(response["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Apple latest news investment stock finance earnings valuation'}`


Find answers with millions of other Apple users in our vibrant community. Search discussions or ask a question about your … Oct 10, 2024 · Change your Apple Account password - Apple Support This is how you change your Apple ID password on your iPhone, iPad, iPod touch, or Apple Watch: 1. … Dec 31, 2024 · Here is: How to use account recovery when you can’t reset your Apple Account password - Apple Support But if you’ve lost access to enough of the context, … Mar 3, 2025 · Is this text message a scam Apple Approval Notice We have noticed that your Apple iCloud id was recently used at "APPLE STORE In CA" for 143.95, paid by … Jan 4, 2025 · Typically an Apple ID is an email address you have used. If you are currently using a mobile device that is logged into your account, go to Settings and tap your name …

/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Invoking: `duckduckgo_search` with `{'query': 'Apple latest financials news'}`


Find answers with millions of other Apple users in our vibrant community. Search discussions or ask a question about your … Oct 10, 2024 · Change your Apple Account password - Apple Support This is how you change your Apple ID password on your … Dec 31, 2024 · Here is: How to use account recovery when you can’t reset your Apple Account password - Apple Support But if … Mar 3, 2025 · Is this text message a scam Apple Approval Notice We have noticed that your Apple iCloud id was recently used at … Jan 4, 2025 · Typically an Apple ID is an email address you have used. If you are currently using a mobile device that is …

/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/local_disk0/.ephemeral_nfs/envs/pythonEnv-116e2167-2376-4d59-aeb4-d03fa4e816c2/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(



Invoking: `youtube_search` with `{'query': 'Apple stock finance investment earnings valuation'}`


['https://www.youtube.com/watch?v=VbzZ6c5Epbw&pp=ygUxQXBwbGUgc3RvY2sgZmluYW5jZSBpbnZlc3RtZW50IGVhcm5pbmdzIHZhbHVhdGlvbg%3D%3D', 'https://www.youtube.com/watch?v=pdm4IVdf5_Y&pp=ygUxQXBwbGUgc3RvY2sgZmluYW5jZSBpbnZlc3RtZW50IGVhcm5pbmdzIHZhbHVhdGlvbg%3D%3D']<p>To get an overview of the latest news about Apple, a search was conducted, but the results did not provide any relevant financial information. However, two YouTube videos were found that review Apple's latest financials: https://www.youtube.com/watch?v=VbzZ6c5Epbw and https://www.youtube.com/watch?v=pdm4IVdf5_Y. These videos may provide valuable insights into Apple's current financial situation and valuation.</p>

> Finished chain.


To get an overview of the latest news about Apple, a search was conducted, but the results did not provide any relevant financial information. However, two YouTube videos were found that review Apple's latest financials: https://www.youtube.com/watch?v=VbzZ6c5Epbw and https://www.youtube.com/watch?v=pdm4IVdf5_Y. These videos may provide valuable insights into Apple's current financial situation and valuation.

Trace(request_id=tr-a67f2c0ec7c7464e961b41b1c7226872)